In [0]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [0]:
import timeit
import array
import os
import collections
import dis
import bisect
import random
import numpy

## 2.0 序列构成的数组

- Python 从 ABC 语言那里继承了用统一的风格去处理序列数据的特点
- Python 中的字符串、列表、字节序列、数组、XML 元素、数据库查询结果等数据结构都共用一套丰富的操作：
  - 迭代、切片、排序、拼接

## 2.1 内置序列类型概览

- Python 标准库用 C 实现了丰富的序列类型
  1. 容器序列
    - `list`，`tuple` 和 `collections.deque`
    - 这些序列能用来存放在不同类型的数据
    - 其中存放的是它们所包含的对任意类型对象的引用
  2. 扁平序列
    - `str`，`bytes`，`bytearray`，`memoryview` 和 `array.array`
    - 这类序列只能容纳一种类型
    - 其中存放的时值而不是引用
    - 扁平序列其实是一段连续的内存空间
    - 体积更小，速度更快，只能存放原子性的数据
    - 不能嵌套使用

- 也可按照序列类型能否被修改来分类
  1. 可变序列 (MutableSequence)
    - `list`， `bytearray`，`array.array`，`collections.deque` 和 `memoryview`
  2. 不可变序列 (Sequence)
    - `tuple`，`str` 和 `bytes`
  - 可变序列与不可变序列的关系图
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200121125821.png width=600>

## 2.2 列表推导和生成器表达式

- 列表推导是构建列表(list)的快捷方式
- 生成器表达式可以用来创建其它任何类型序列

### 2.2.1 列表推导和可读性

###### 示例2-1

In [0]:
symbols = "$¢£¥€¤"
codes = []
for symbol in symbols:
  codes.append(ord(symbol))

codes

###### 示例 2-2

In [0]:
symbols = "$¢£¥€¤"
codes = [ord(symbol) for symbol in symbols]
codes

###### 列表推导使用的基本原则

- 只用列表推导来创建新的列表，并且尽量保持简短
- 如果列表推导的代码超过两行，可能需要采用 `for` **循环重写**

###### 句法提示

- python 会忽略代码里 `[]`、`{}` 和`()` 中的换行
- 如果代码中有多行的列表、列表推导式、生成器表达式、字典等，可以省略续行符

###### 列表推导不再存在内存泄露的问题

- 在 Python3 中，列表推导、生成器表达式，以及同它们很相似的集合(`set`)推导和字典(`dict`)推导都有自己的局部作用域，与函数类似

In [0]:
x = 'ABC'
dummy = [ord(x) for x in x]
x

In [0]:
dummy

### 2.2.2 列表推导同 `filter` 和 `map` 的比较

- 列表推导可以把一个序列或是其它可迭代类型中的元素过滤或是加工，然后再新建一个列表

###### 示例 2-3 

In [0]:
symbols = "$¢£¥€¤"
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
beyond_ascii

In [0]:
beyond_ascii = list(filter(lambda c: c>127, map(ord, symbols)))
beyond_ascii

- 对于示例 2-3 来说，其中列表推导式的运行速度要快于 `filter/map` 组合

In [0]:
TIMES = 10000

SETUP = """
symbols = '$¢£¥€¤'
def non_ascii(c):
    return c > 127
"""

def clock(label, cmd):
    res = timeit.repeat(cmd, setup=SETUP, number=TIMES)
    print(label, *('{:.3f}'.format(x) for x in res))

clock('listcomp        :', '[ord(s) for s in symbols if ord(s) > 127]')
clock('listcomp + func :', '[ord(s) for s in symbols if non_ascii(ord(s))]')
clock('filter + lambda :', 'list(filter(lambda c: c > 127, map(ord, symbols)))')
clock('filter + func   :', 'list(filter(non_ascii, map(ord, symbols)))')

### 2.2.3 笛卡尔积

- 两个或两个以上列表中的元素对构成元组，这些元组构成的列表就是笛卡尔积
  - 示意图
    - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200121135000.png width=600>

###### 示例 2-4 使用列表推导计算笛卡尔积

In [0]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
tshirts = [(color, size)  for color in colors
                for size in sizes ]
tshirts

In [0]:
tshirts = [(color, size) for size in sizes
               for color in colors]
tshirts

### 2.2.4 生成器表达式

- 列表推导式的作用只有一个：生成列表。如果想生成其它类型的序列，则需要使用生成器表达式
- 虽然列表推导也可用来初始化元组、数组或其他类型的序列类型，但是生成器表达式是更好的选择，因为：
  - 生成器表达背后遵循了迭代器协议
  - 生成器可以逐个的产生元素，而不是先建立一个完整的列表，如此能够节约内存
- 生成器表达式的语法跟列表推导差不多，只不过需要将方括号换为元括号

###### 示例 2-5 用生成器来初始化元组和数组

- 如果生成器表达式是一个函数调用过程中唯一的参数，则可以省略括号

In [0]:
symbols = "$¢£¥€¤"
tuple(ord(symbol) for symbol in symbols)

- 如果函数需要两个参数，则生成器的括号是必需的

In [0]:
array.array('I', (ord(symbol) for symbol in symbols))

###### 示例 2-6 使用生成器来计算笛卡尔积

In [0]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
for tshirt in ('%s %s'%(c, s) for c in colors
                   for s in sizes):
  print(tshirt)

## 2.3 元组不仅仅是不可变的列表

- 元组除了可以用作不可变列表，还可以用于没有字段名的记录

- 列表中的元素，最好有一些通用的特性
  - 如果列表中的东西不能比较大小，则就不能对列表进行排序
- 元组则恰与列表相反，其经常用来存放彼此之间没有关系的数据记录

In [0]:
l = [28, 14, '28', 5, '9', '1', 0, 6, '23', 19]
sorted(l)

TypeError: ignored

### 2.3.1 元组和记录

- 元组其实是对数据的记录
- 如果把元组当做一些字段的集合，那么数量和位置信息就会变得比较重要

 ###### 示例 2-7 把元组用作记录

In [0]:
lax_coordinates = (33.9425, -118.408056)
city, year, pop, chg, area = ('Tokyo', 2003, 32450, 0.66, 8014)
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP', 'XDA295856')]

In [0]:
for passport in sorted(traveler_ids):
  print("%s/%s" % passport)

BRA/CE342567
ESP/XDA295856
USA/31195855


- 拆包时可以使用占位符来怱略特定位置的元素

In [0]:
for country, _ in traveler_ids:
  print(country)

USA
BRA
ESP


### 2.3.2 元组拆包

- 拆包让元组可以完美地当作记录来使用
- 元组拆包也常被叫做[可迭代元素拆包](https://www.python.org/dev/peps/pep-3132/)

###### 元组拆包常见的形式是平行赋值

In [0]:
lax_coordinates = (33.9425, -118.408056)
latitude, longtitude = lax_coordinates
latitude

33.9425

In [0]:
longtitude

-118.408056

###### 元组拆包可以不使用中间变量交换两个变量的值

In [0]:
a = 6
b = 8
a, b = b, a

In [0]:
a

8

In [0]:
b

6

###### 使用 \* 号运算符可以把一个可迭代对象拆开作为函数的参数

In [0]:
divmod(20, 8)

(2, 4)

In [0]:
t = (20, 8)
divmod(*t)

(2, 4)

In [0]:
quotient, remainder = divmod(*t)
quotient, remainder

(2, 4)

###### 函数可以以元组的形式返回多个值，然后可以借助拆包才取出特定的值

In [0]:
_, filename = os.path.split('/home/luciano/.ssh/idrsa.pub')
filename

'idrsa.pub'

#### 用 \* 来处理剩下的元素
- 函数可以用 $*args$ 来获取不定数量的参数，在 Python3 中，这个概念被扩展到了平行赋值中

In [0]:
a, b, *rest = range(5)
a, b, rest

(0, 1, [2, 3, 4])

In [0]:
a, b, *rest = range(3)
a, b, rest

(0, 1, [2])

In [0]:
a, b, *rest = range(2)
a, b, rest

(0, 1, [])

###### \* 前缀只能用在一个变量前面，但是这个变量可以出现在赋值表达式的任意位置

In [0]:
a, *body, c, d = range(5)
a, body, c, d

(0, [1, 2], 3, 4)

In [0]:
*head, b, c, d = range(5)
head, b, c, d

([0, 1], 2, 3, 4)

### 2.3.3 嵌套元组拆包 

- 接受表达式的元组可以是嵌套式的

In [0]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

print('{:15}|{:^9}|{:^9}'.format('', 'lat.', 'long.'))
fmt = '{:15}|{:^9.4f}|{:^9.4f}'
for name, cc, pop, (latitude, longtitude) in metro_areas:
  if longtitude <= 0:
    print(fmt.format(name, latitude, longtitude))

               |  lat.   |  long.  
Mexico City    | 19.4333 |-99.1333 
New York-Newark| 40.8086 |-74.0204 
Sao Paulo      |-23.5478 |-46.6358 


### 2.3.4 具名元组

- `collections.namedtuple` 是一个工厂函数，可以用来构建一个带字段名的元组和一个有名字的类
- 用 `namedtuple` 构建的类的实例消耗的内存与元组是一样的，因为字段名都被存放在对应的类里
  - 这个实例跟普通的对象实例比起来要小一些，因为 Python 不会用 `__dict__` 来存放这些实例的属性

###### 示例 2-9 定义和使用具名元组

In [0]:
City = collections.namedtuple('City', 'name, country population coordinates')
tokyo = City('Tokyo', 'JP', '39.933', (35.689722, 139.691667))
tokyo

City(name='Tokyo', country='JP', population='39.933', coordinates=(35.689722, 139.691667))

In [0]:
tokyo.population

'39.933'

In [0]:
tokyo.coordinates

(35.689722, 139.691667)

In [0]:
tokyo[1]

'JP'

##### 示例 2-10 具名元素的属性和方法

- `_fields` 属性是一个包含这个类所有字段名称的元组
- `_make()` 通过接受一个可迭代的对象来生成这个类的一个实例，作用跟 `City(*delhi_data)` 是一样的
- `_asdict()` 把具名元素以 `collections.OrderedDict` 的形式返回，可以利用它把元组中的信息友好的呈现出来

In [0]:
City._fields

('name', 'country', 'population', 'coordinates')

In [0]:
LatLong = collections.namedtuple('LatLong', 'lat long')
delhi_data = ('Delhi NCR', 'IN', '21.935', LatLong(28.613889, 77.208889))
delhi = City._make(delhi_data)
delhi._asdict()

OrderedDict([('name', 'Delhi NCR'),
             ('country', 'IN'),
             ('population', '21.935'),
             ('coordinates', LatLong(lat=28.613889, long=77.208889))])

In [0]:
for key, value in delhi._asdict().items():
  print(key + ':', value)

name: Delhi NCR
country: IN
population: 21.935
coordinates: LatLong(lat=28.613889, long=77.208889)


### 2.3.5 作为不可变列表的元组

###### 表2-1：列表或元组的方法和属性

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200122160925.png width=600>

## 2.4 切片

### 2.4.1 为什么切片和区间分忽略最后一个元素

- 好处
  1. 当只有最后一个位置信息时，也可以快速看出切片和区间里有几个元素
    - `range(3)` 和 `my_list[:3]` 都返回 3 个元素
  2. 当起止位置信息都可见时，可以快速计算出切片和区间的长度
    - `stop - start` 即为区间的长度
  3. 可以例用任一个下标把序列分割成不重叠的两部分
    - 写成 `my_list[:x]` 和 `my_list[x:]` 即可实现


### 2.4.2 对对象进行切片

###### $s[a:b:c]$ 对 $s$ 在 $a$ 和 $b$ 之间以 $c$ 为间隔取值

In [0]:
s = 'bicycle'
s[::3]

'bye'

In [0]:
s[::-1]

'elcycib'

In [0]:
s[::-2]

'eccb'

###### 切片对象

- `a:b:c` 这种用法实际会返回一个切片对象：`slice(a, b, c)`
- 在对 `seq[start:stop:step]` 进行求值的时候， Python 实际上会调用 `seq.__getitem__(slice(start, stop, step))`

###### 示例 2-11 纯文本文件形式的收据以一行字符串的形式被解析

In [0]:
invoice = """
0.....6.................................40........52...55........
1909  Pimoroni PiBrella                 $17.50       3   $52.50
1489  6mm Tactile Switch x20             $4.95       2   $9.90
1510  Panavise Jr. - PV-201             $28.00       1   $28.00
1601  PiTFT Mini Kit 320x240            $34.95       1   $34.95"""
SKU = slice(0, 6)
DESCRIPTION = slice(6, 40)
UNIT_PRICE = slice(40, 52)
QUANTITY = slice(52, 55)
ITEM_TOTAL = slice(55, None)
line_items = invoice.split('\n')[2:]
for item in line_items:
  print(item[UNIT_PRICE], item[DESCRIPTION])

$17.50       Pimoroni PiBrella                 
 $4.95       6mm Tactile Switch x20            
$28.00       Panavise Jr. - PV-201             
$34.95       PiTFT Mini Kit 320x240            


### 2.4.3 多维切片和省略

#### 多维切片

- `[]` 中可以使用逗号分开的多个索引或切片
- 二维的 `numpy.ndarray` 可以用 `a[i, j]` 的形式获取，或者使用 `a[m:n, k:l]` 的方式获得二维切片
- 为了实现多维切片， `__getitem__` 和 `__setitem__` 需要以元组的形式接收 `a[i, j]` 中的索引参数
  - 为了获得 `a[i, j]` 的值， Python 需要调用 `a.__getitem__((i, j))`


#### 省略号

- 省略号的正确写法是 `...`，其是 `Ellipsis` 对象的别名，`Ellipsis` 对象是 `ellipsis` 类的单一实例
- 省略号为切片规范的一部分
  - `f(a, ..., z)` 或 `a[i:...]`
  - 如果 `x` 是四维数组，则 `x[i,...]` 相当于 `x[i, :, :, :]`

### 2.4.4 给切片赋值

- 如果把切片放在赋值语名左边，或把它作为 `del` 的对象，就可以对序列进行嫁接、切除或就地修改操作

###### 代码示例

In [0]:
l = list(range(10))
l

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [0]:
l[2:5] = [20, 30]
l

[0, 1, 20, 30, 5, 6, 7, 8, 9]

In [0]:
del l[5:7]
l

[0, 1, 20, 30, 5, 8, 9]

In [0]:
l[3::2] = [11, 22]
l

[0, 1, 20, 11, 5, 22, 9]

- 如果赋值对象是切片，那么赋值语句右侧必须是可迭代对象。即使只有单独一个值，也需将其转换为可迭代序列

In [0]:
l[2:5] = 100

TypeError: ignored

In [0]:
l[2:5] = [100]
l

[0, 1, 100, 22, 9]

## 2.5 对序列使用 `+` 和 `*`

- `+` 两侧的序列由相同的数据类型构成，在拼接的过程中，两个被操作的序列均不会被修改，而是构建一个全新的序列
- `*` 也不会修改被操作的序列，而是构建一个全新的序列

In [0]:
l = [1, 2, 3]
l * 3

[1, 2, 3, 1, 2, 3, 1, 2, 3]

In [0]:
5 * 'abcd'

'abcdabcdabcdabcdabcd'

### 建立由列表组成的列表

- `a*n` 中如果 `a` 中的元素是引用，需要格外注意
  - `mylist=[[]]* 3` 是包含 3 个引用的列表，而且这 3 个引用指向的是同一个列表

###### 示例2-12： 一个包含 3 个列表的列表

In [0]:
board = [['-']*3 for i in range(3)]
board

[['-', '-', '-'], ['-', '-', '-'], ['-', '-', '-']]

In [0]:
board[1][2] = 'X'
board

[['-', '-', '-'], ['-', '-', 'X'], ['-', '-', '-']]

###### 示例 2-13： 含有 3 个指向同一对象的引用的列表

In [0]:
weird_board = [['_'] * 3] * 3
weird_board

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]

In [0]:
weird_board[1][2] = '0'
weird_board

[['_', '_', '0'], ['_', '_', '0'], ['_', '_', '0']]

## 2.6 序列的增量赋值

- 增量赋值运算符(`+=`, `*=`) 取决于它们的第一个操作对象
- `a += b`
  - 如果 `a` 实现了 `__iadd__` 方法，就会调用这个方法，同时 `a` 会就地改动，与 `a.extend(b)` 的作用相同
  - 如果 `a` 没有实现 `__iadd__` 方法，则会调用 `__add__` 方法，相当于调用 `a = a + b`， 此时会先得到一个新对象
  然后赋值给 `a`
- 可变序列一般实现了 `__iadd__` 方法， `+=` 即为就地加法
- 不可变序列一般没有实现 `__iadd__` 方法， `+=` 等效于普通的加法


###### 代码实现

In [0]:
l = [1, 2, 3]
id(l)

140551689529928

In [0]:
l *= 2
l

[1, 2, 3, 1, 2, 3]

In [0]:
id(l)

140551689529928

In [0]:
t = (1, 2, 3)
id(t)

140551690812008

In [0]:
t *= 2
t

(1, 2, 3, 1, 2, 3)

In [0]:
id(t)

140551699349864

### 一个关于 `+=` 的谜题

##### 示例 2-*15：* t[2] 被改动了，但还是有异常抛出

In [0]:
t = (1, 2, [30, 40])
t[2] += [50, 60]

TypeError: ignored

In [0]:
t[2]

[30, 40, 50, 60]

###### [Python Tutor](http://www.pythontutor.com) 中的运行结果

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200217215006.png width=800>

###### 字节码分析

In [0]:
dis.dis('s[a] += b')

  1           0 LOAD_NAME                0 (s)
              2 LOAD_NAME                1 (a)
              4 DUP_TOP_TWO
              6 BINARY_SUBSCR
              8 LOAD_NAME                2 (b)
             10 INPLACE_ADD
             12 ROT_THREE
             14 STORE_SUBSCR
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


- `BINARY_SUBSCR`: 将 `s[a]` 的存入 `TOS` 中 （Top Of Stack，栈的顶端）
- `INPLACE_ADD`: 计算 `TOS += b`，这一步能够完成的原因是 `TOS` 指向可变对象
- `STORE_SUBSCR`: `s[a] = TOS` 赋值，这一步会报错，因为 `s` 为不可变的元组

###### 3 个经验

- 元组中不要存放可变对象
- 增量赋值不是一个原子操作
- dis.dis 可以查看 ptyhon 语句的字节码，有助于理解 Python 背后的运行原理

## 2.7 `list.sort` 方法和内置函数 `sorted`

##### 返回值的区别

- `list_sort` 会就地排序算法，其返回值为 `None`
  - Python 中的惯例是如果一个函数或者方法对对象进行的是就地改动，则其应该返回 `None`，好让调用者知道
  传入的参数发生了改变
- 内置函数 `sorted` 会新建一个列表作为返回值
  - 返回一个新对象的方法，可以串联起来使用，从而形成连贯接口([fluent interface](https://en.wikipedia.org/wiki/Fluent_interface)) 

##### 两个可选的关键字参数

- `reverse`
  - 如果设置为 `True`，被排序的序列里的元素会以降序输出，默认值为 `False`
- `key`
  - 一个只有一个参数的函数，这个函数会被应用到序列中的每一个元素上，所产生的结果将是排序算法依赖的对比
  关键字。
    - 对于字符串排序，可以用 `key=str.lower` 来实现忽略大小写的操作，使用 `key=len` 进行基于字符串
    长度的排序
  - 参数的默认值为恒等函数(identity function)，即默认用元素本身的值来进行排序
  - `key` 还可以用于内置函数 `min()`, `max()` 中

##### 代码示例

In [0]:
fruits = ['grape', 'raspberry', 'apple', 'banana']
sorted(fruits)

['apple', 'banana', 'grape', 'raspberry']

In [0]:
fruits

['grape', 'raspberry', 'apple', 'banana']

In [0]:
sorted(fruits, reverse=True)

['raspberry', 'grape', 'banana', 'apple']

In [0]:
sorted(fruits, key=len)

['grape', 'apple', 'banana', 'raspberry']

In [0]:
sorted(fruits, key=len, reverse=True)

['raspberry', 'banana', 'grape', 'apple']

In [0]:
fruits

['grape', 'raspberry', 'apple', 'banana']

In [0]:
fruits.sort()

In [0]:
fruits

['apple', 'banana', 'grape', 'raspberry']

## 2.8 用 `bisect` 来管理已排序的序列

- `bisect` 包含两个主要函数，`bisect` 和 `insort`，两个函数利用二分查找算法来在有序序列中查找或插入元素

### 2.8.1 用 `bisect` 来搜索

- `bisect(haystack, ndeeld)` 在 `haystack` 里搜索 `neddle` 的位置$x$
  - 把 `neddle` 插入位置 $x$ 后， `haystack` 还能保持升序
- `bisect` 函数是 `bisect_right` 函数的别名，返回的是跟它相等的元素之后的位置
- `bisect_left` 返回的位置是跟被插入元素相等的元素的位置

#### 示例 2-17 在有序数组中用 `bisect` 查找某个元素的插入位置

##### bisect_demo.py 的文件内容

In [0]:
%%writefile ch2/bisect_demo.py
# BEGIN BISECT_DEMO
import bisect
import sys

HAYSTACK = [1, 4, 5, 6, 8, 12, 15, 20, 21, 23, 23, 26, 29, 30]
NEEDLES = [0, 1, 2, 5, 8, 10, 22, 23, 29, 30, 31]

ROW_FMT = '{0:2d} @ {1:2d}    {2}{0:<2d}'

def demo(bisect_fn):
    for needle in reversed(NEEDLES):
        position = bisect_fn(HAYSTACK, needle)  # <1>
        offset = position * '  |'  # <2>
        print(ROW_FMT.format(needle, position, offset))  # <3>

if __name__ == '__main__':

    if sys.argv[-1] == 'left':    # <4>
        bisect_fn = bisect.bisect_left
    else:
        bisect_fn = bisect.bisect

    print('DEMO:', bisect_fn.__name__)  # <5>
    print('haystack ->', ' '.join('%2d' % n for n in HAYSTACK))
    demo(bisect_fn)

# END BISECT_DEMO

Overwriting ch2/bisect_demo.py


In [0]:
!python ch2/bisect_demo.py

DEMO: bisect
haystack ->  1  4  5  6  8 12 15 20 21 23 23 26 29 30
31 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |31
30 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |30
29 @ 13      |  |  |  |  |  |  |  |  |  |  |  |  |29
23 @ 11      |  |  |  |  |  |  |  |  |  |  |23
22 @  9      |  |  |  |  |  |  |  |  |22
10 @  5      |  |  |  |  |10
 8 @  5      |  |  |  |  |8 
 5 @  3      |  |  |5 
 2 @  1      |2 
 1 @  1      |1 
 0 @  0    0 


In [0]:
!python ch2/bisect_demo.py left

DEMO: bisect_left
haystack ->  1  4  5  6  8 12 15 20 21 23 23 26 29 30
31 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |31
30 @ 13      |  |  |  |  |  |  |  |  |  |  |  |  |30
29 @ 12      |  |  |  |  |  |  |  |  |  |  |  |29
23 @  9      |  |  |  |  |  |  |  |  |23
22 @  9      |  |  |  |  |  |  |  |  |22
10 @  5      |  |  |  |  |10
 8 @  4      |  |  |  |8 
 5 @  2      |  |5 
 2 @  1      |2 
 1 @  0    1 
 0 @  0    0 


#### 示例 2-18  根据一个分数，找到它所对应的成绩

In [0]:
def grade(score, breakpoints=[60, 70, 80, 90], grades='FDCBA'):
  return grades[bisect.bisect(breakpoints, score)]

In [0]:
[grade(score) for score in [33, 99, 77, 70, 89, 90, 100]]

['F', 'A', 'C', 'C', 'B', 'A', 'A']

### 2.8.2 用 `bisect.insort` 插入新元素

- `insort(seq, item)` 将变量 `item` 插入到已排序序列 `seq` 中，并保持 `seq` 的升序

#### 示例 2-19 

###### bisect_insort.py 的源码

In [0]:
%%writefile ch2/bisect_insort.py
import bisect
import random

SIZE = 7

random.seed(1729)

my_list = []
for i in range(SIZE):
  new_item = random.randrange(SIZE*2)
  bisect.insort(my_list, new_item)
  print("%2d -> " % new_item, my_list)

Overwriting ch2/bisect_insort.py


In [0]:
!python ch2/bisect_insort.py

10 ->  [10]
 0 ->  [0, 10]
 6 ->  [0, 6, 10]
 8 ->  [0, 6, 8, 10]
 7 ->  [0, 6, 7, 8, 10]
 2 ->  [0, 2, 6, 7, 8, 10]
10 ->  [0, 2, 6, 7, 8, 10, 10]


## 2.9 当列表不是首选时

### 2.9.1 数组

- 如果只是一个包含数字的列表，则 `array.array` 比 `list` 更加高效
  - 数组的背后并不是 `float` 对象，而是数字的机器翻译
- 数组支持所有跟可变序列有关的操作
- 数组还提供从文件读取和存入文件更快的方法
  - 如 `.frombytes` 和 `.tofile`
- Python 创建数组还需要一个类型码，这个类型码与 C 语言的数据类型相似

#### 示例 2-20 一个浮点型数组的创建、存入文件和文件读取过程

In [0]:
floats = array.array('d', (random.random() for i in range(10 ** 7)))
floats[-1]

0.5671106664942266

In [0]:
with open('ch2/floats.bin', 'wb') as fp:
  floats.tofile(fp)

In [0]:
floats2 = array.array('d')
with open('ch2/floats.bin', 'rb') as fp:
  floats2.fromfile(fp, 10**7)
floats2[-1]

0.5671106664942266

In [0]:
floats == floats2

True

#### 表2-2： 列表与数组的属性和方法

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/t2020227.png width=800>

#### 排序问题

- 从 Python3.4 开始，数组不再支持 `list.sort()` 这种就地排序方法。如果要给数组排序，需要用 `sorted`
函数新建一个数组。
```python
a = array.array(a.typecode, sorted(a))
```
  - 因为 `sorted()` 返回的是一个列表


### 2.9.2 内存视图

- 内存视图 (memoryview) 其实是泛化和去数学化的 Numpy 数组，可以在不复制内容的前提下，在数据结构之间
共享内存
  - 数据结构可以是任何形式，比如 PIL 图片， SQLite 数据库和 Numpy 数组等
  - 此功能在处理大型数据集合时非常重要

- `memoryview.cast` 能用不同的方式读写同一块内存数据，与 C 语言中的类型转换概念相似。其会将同一块内
存里的内容打包成一个全新的 `memoryview` 对象返回

###### `memoryview.cast` 支持的数据类型

| 格式符       | C语言类型              | Python类型           | Standard size |
|-----------|--------------------|--------------------|---------------|
| x         | pad byte\(填充字节\)   | no value           |               |
| c         | char               | string of length 1 | 1             |
| b         | signed char        | integer            | 1             |
| B         | unsigned char      | integer            | 1             |
| ?         | \_Bool             | bool               | 1             |
| h         | short              | integer            | 2             |
| H         | unsigned short     | integer            | 2             |
| i         | int                | integer            | 4             |
| I\(大写的i\) | unsigned int       | integer            | 4             |
| l\(小写的L\) | long               | integer            | 4             |
| L         | unsigned long      | long               | 4             |
| q         | long long          | long               | 8             |
| Q         | unsigned long long | long               | 8             |
| f         | float              | float              | 4             |
| d         | double             | float              | 8             |
| s         | char\[\]           | string             |               |
| p         | char\[\]           | string             |               |
| P         | void \*            | long               |               |


#### 示例 2-21 通过改变数组中的一个字节来更新数组里面某个元素的值

In [4]:
import array
numbers = array.array('h', [-2, -1, 0, 1, 2])
memv = memoryview(numbers)
len(memv)

5

In [5]:
memv[0]

-2

In [6]:
memv_oct = memv.cast('B')
memv_oct.tolist()  # 低字节在前，高字节在后

[254, 255, 255, 255, 0, 0, 1, 0, 2, 0]

In [8]:
memv_oct[1:3].tolist()

[255, 255]

In [0]:
memv_oct[5] = 4

In [0]:
numbers

array('h', [-2, -1, 1024, 1, 2])

### 2.9.3 Numpy 和 Scipy

- Numpy 实现了多维同质数组和矩阵，这些数据结构不但能处理数字，还能存放其他由用户定义的记录。通过 Numpy，
用户可以对这些数据结构里的元素进行高效操作
- SciPy 是基于 Numpy 的另一个库，其提供了许多跟科学计算相关的算法
  - 专为线性代数，数值积分和统计学设计

#### 示例2-22： 对 `numpy.ndarray` 的行和列进行基本操作

In [0]:
a = numpy.arange(12)
a

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [0]:
type(a)

numpy.ndarray

In [0]:
a.shape

(12,)

In [0]:
a.shape = 3, 4
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [0]:
a[2]

array([ 8,  9, 10, 11])

In [0]:
a[2][1]

9

In [0]:
a[:, 1]

array([1, 5, 9])

In [0]:
a.transpose()

array([[ 0,  4,  8],
       [ 1,  5,  9],
       [ 2,  6, 10],
       [ 3,  7, 11]])

#### `numpy` 写入和读取文件

In [0]:
floats = numpy.loadtxt('ch2/floats-10M-lines.txt')
floats[-3:]

array([56771107.91378655, 22580791.63170634, 92427100.75245066])

In [0]:
floats *= 0.5
floats[-3:]

array([28385553.95689327, 11290395.81585317, 46213550.37622533])

In [0]:
from time import perf_counter as pc
t0 = pc(); floats /= 3 ; pc() - t0

0.020284793001337675

In [0]:
numpy.save("ch2/floats-10M", floats)  # 储存为后缀为 .npy 的二进制文件

In [0]:
floats2 = numpy.load("ch2/floats-10M.npy", 'r+')  # 使用了内存映射的机制，允许在内存不足的情况下对数组进行切片操作
floats2 *= 6
floats2[-3:]

memmap([56771107.91378655, 22580791.63170634, 92427100.75245066])

#### 其它常用库

- [pandas](http://pandas.pydata.org) 和 [Blaze](http://blaze.pydata.org) 数据分析库即以 numpy 和 
pandas 为基础
  - 其提供了高效的且能储存非数值类型数据的数组类型和读写常见数据文件格式(csv, xls, SQL, HDF5)的功能

### 2.9.4 双向队列和其他形式的队列

- `collections.deque` 类(双向队列)是一个线程安全、可以快速从两端添加或者删除元素的数据类型
  - `append` 和 `popleft` 是原子操作，可以在多线程程序中安全地当作队列使用，不用担心资源锁的问题

#### 示例 2-23 使用双向队列

In [0]:
dq = collections.deque(range(10), maxlen=10)  # maxlen 为最大能容纳的元素数，一旦设定，便不能再更改

In [0]:
dq

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [0]:
dq.rotate(3)  # 队列的旋转操作，当 n > 0时，队列最右边的 n　个元素会移动左边； n < 0 时，队列最左边的 n 个元素会移动到右边
dq

deque([7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [0]:
dq.rotate(-4)
dq

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 0])

In [0]:
dq.appendleft(-1)  # 从左边加入元素，当 deque 满时，其尾部的元素会被删除掉
dq

deque([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [0]:
dq.extend([11, 22, 33])  # 由于 deque 已满，所以在尾部添加元素，会导致头部的元素被删除
dq

deque([3, 4, 5, 6, 7, 8, 9, 11, 22, 33])

In [0]:
dq.extendleft([10, 20, 30, 40])
dq

deque([40, 30, 20, 10, 3, 4, 5, 6, 7, 8])

- `extendleft(iter)` 方法会把迭代器中的元素逐个添加到双向队列的左边，因此迭代器中的元素逆序出现在队列里

#### 列表和双向队列的方法

- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200227122636.png width=800>

#### 其他 Python 标准库中的类

###### `queue`

- 提供了同步（线程安全) 类， `Queue, LifoQueue` 和 `PriorityQueueu`，不同的线程可以用这些数据类型交
换信息
- 这三个类均有一个可选参数 `maxsize`，用来限定队列的大小，当满员时，其不会丢弃旧的元素，而是会被锁住
  - 这一特性适合用来控制活跃线程的数量

###### `multiprocessing`

- `Queue`, 与 `queue.Queue` 类似，设计给进程间通信使用
- `multiprocesssing.JoinableQueue` 类，可以让任务管理更加方便

###### `asyncio`

- 有 `Queue, LifoQueue, PriorityQueue` 和 `JoinableQueue`，这些类为异步编程里的任务管理，提供了专门
的便利

###### `heapq`

- 没有队列类，但是提供了 `heappush` 和 `heapop` 方法，从而可以将可变序列当做堆队列或优先队列来使用